In [1]:
# load necessary packages
import pandas as pd                 # dataframe storing
from bs4 import BeautifulSoup       # parsing web content in a nice way
import ssl                          # MAY BE UNNECESSARY: provides access to the security socket layer (ssl) https://docs.python.org/3/library/ssl.html
import urllib                       # open and navigate URL's


In [2]:
# This is a dummy URL that links to the eHRAF page. 
# Later I will set this up to allow for modular querying
# Search for Apple in PSF
url = 'https://ehrafworldcultures.yale.edu/search?q=text%3AApple&fq=culture_level_samples%7CPSF'

In [13]:


# read in the url (timeout set at 5 seconds as precuation but is optional)
#read the response into soup for web parsing
with urllib.request.urlopen(url, context=ssl.SSLContext(ssl.PROTOCOL_TLS), timeout = 5) as response:
    soup = BeautifulSoup(response.read())



/var/folders/r8/j6vx966d5rj6srtb4x895kvr0000gp/T/ipykernel_88084/3902438760.py:1: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  with urllib.request.urlopen(url, context=ssl.SSLContext(ssl.PROTOCOL_TLS), timeout = 5) as response:


In [16]:
import bs4.element as element       # set as element for easier indexing through the webpage


web_body_child_tags = list(filter(lambda x: type(x) == element.Tag, soup.body.children))
len(web_body_child_tags)
for x in web_body_child_tags:
    print(x.name)

div
script


In [20]:
for x in soup.body.div.children:
    print(x.name)

div


In [46]:
tables_more_specific = soup.find_all('div',
{'class':'results__container hierarchy'})
tables_more_specific

# for x in tables_more_specific:
#     for y in x.children:
#         print(y.name)

[<div class="results__container hierarchy"><section class="mdc-card not-found"><div class="mdc-card__media"><h2 class="mdc-typography--headline5">Viewing Your Search Results</h2><p>Once you have submitted a search, your results will display in the lower half of the page.</p><p>You can also explore your results in one of the following ways using the <strong>Insights</strong> options near the top of the page:</p><ul><li><strong> Table Overview</strong>: Displays the total number of results by regions, subregions, cultures, documents, and paragraphs.</li><li><strong>Cloud Overview</strong>: Represents the subjects associated with your search in a word cloud.</li><li><strong>Map Overview</strong>: Pinpoints the geographic location of cultures associate with your search.</li></ul></div></section></div>]

In [ ]:
#REDO all over again, let's try using selenium. If you are reading this and the above section is not deleted... delete it


In [2]:
import pandas as pd                 # dataframe storing
from bs4 import BeautifulSoup       # parsing web content in a nice way
import ssl                          # MAY BE UNNECESSARY: provides access to the security socket layer (ssl) https://docs.python.org/3/library/ssl.html
import urllib                       # open and navigate URL's


from selenium import webdriver      # load and run the webpage dynamically.
#from selenium.webdriver.chrome.options import Options



In [43]:
# Use a "headless" Chrome page to dynamically load the page for scraping. 
# Requires webdriver to be downloaded and then its path directed to
path = "/Users/ericchantland/Documents/GitHub/Boyer_eHRAF_Misery/Chromedriver"
driver = webdriver.Chrome(executable_path = path)


# Load the HTML page
driver.get('https://ehrafworldcultures.yale.edu/search?q=text%3AApple&fq=culture_level_samples%7CPSF')

# Find the click on each tab to reveal content for scraping
# Elements must be individually clicked backwards. I do not know why this is a thing but my guess is each 
# clicked tab adds HTML and makes some indexing no longer point to retrived tab. Loading backwards avoids this.
country_tab = driver.find_elements_by_class_name('trad-overview__result')
for i in range(len(country_tab)-1,-1,-1):
    try:
        country_tab[i].click()
    except:
        print(f"WARNING tab {i} failed to be clicked")

# Parse processed webpage with BeautifulSoup
soup = BeautifulSoup(driver.page_source)



In [20]:
#Example of finding then printing out the dynamic webpage (before clicking). Shown here are the continents
continent_dir = soup.find_all('div',
{'class':'trad-overview__result'})

for x in continent_dir:
    print(x.h4.button.text)

### FOR REFERENCE, DELETE
# results_container = tables_more_specific[0].children

# for x in results_container:
#     print(x.name)

Africa
Asia
Europe
Middle America and the Caribbean
Middle East
North America
Oceania
South America


In [21]:
continent_dir = soup.find_all('div',
{'class':'trad-overview__table'})
len(continent_dir)
#mdc-data-table__content

1